In [1]:
'''Interactive run file for the G2-structure learning'''
#...ensure this notebook is using the correct kernel for your virtual environment
# Import libraries
import os
import yaml
import numpy as np
import tensorflow as tf

# Import functions
from models.model import GlobalModel
from sampling.sampling import LinkSample
from geometry.compression import form_to_vec, vec_to_form, vec_to_metric
from geometry.geometry import exterior_derivative
from geometry.wedge_product import wedge_product
from geometry.patches import patch_indices_to_scalar

# Print the hyperparameters
with open(os.getcwd()+'/hyperparameters/hps.yaml', "r") as file:
    hp = yaml.safe_load(file)
print(f'Run hyperparameters:\t...edit in hyperparameters/hps.yaml\n{hp}')

Run hyperparameters:	...edit in hyperparameters/hps.yaml
{'metric': False, 'num_samples': 200000, 'saved_model': False, 'saved_model_path': '...', 'n_hidden': 64, 'n_layers': 3, 'activations': 'gelu', 'use_bias': True, 'embedding_dim': 8, 'epochs': 200, 'batch_size': 100, 'init_learning_rate': 0.005, 'min_learning_rate': 0.005, 'validate': False, 'val_print': False, 'num_val_samples': 500, 'val_batch_size': 100, 'verbosity': 1, 'print_losses': False, 'print_interval': 1}


In [ ]:
# Run the training script
model_name = 'test_pe'
!python3 -m run {model_name}

In [2]:
# Import the trained model
#loaded_model_name = model_name #...or set to desired name if not model trained above
#loaded_model_path = os.getcwd() + f"/runs/link_model_{loaded_model_name}.keras" #...reimport the model
loaded_model_path = os.getcwd() + f"/models/link_models/link_model_3form.keras" #...import a pre-trained model instead

# Load the model
loaded_model = tf.keras.models.load_model(loaded_model_path, custom_objects={'GlobalModel': GlobalModel})


absl:WARNING:At this time, the v2.11+ optimizer `tf.keras.optimizers.Adam` runs slowly on M1/M2 Macs, please use the legacy Keras optimizer instead, located at `tf.keras.optimizers.legacy.Adam`.


In [3]:
# Test loaded model on fake link points
# Testing hyperparameters
testsize = 1000 #...how many link points to use in the testing

# Generate the testing data on the Link
test_dataset = LinkSample(testsize)
test_linkpts = test_dataset.link_points
test_patch_idxs = patch_indices_to_scalar(test_dataset.one_idxs, test_dataset.dropped_idxs)
if not hp["metric"]:
    test_link_outputs = test_dataset.g2_form
else:
    test_link_outputs = test_dataset.g2_metric
test_linkpts_tf = tf.convert_to_tensor(test_linkpts)
test_link_outputs_tf = tf.convert_to_tensor(test_link_outputs)
    
# Compute the NN test outputs      
if not hp["metric"]:
    # Compute the predicted G2 3-forms
    predicted_g2form_vecs = np.array(loaded_model.model.predict([test_linkpts_tf, test_patch_idxs]))
    predicted_g2forms = np.array(vec_to_form(predicted_g2form_vecs, 7, 3).numpy())
    print(f'G2 3-forms computed... (shape: {predicted_g2forms.shape})')
else:
    # Compute the predicted G2 metrics
    predicted_g2metric_vecs = np.array(loaded_model.model.predict([test_linkpts_tf, test_patch_idxs]))
    predicted_g2metrics = np.array(vec_to_metric(predicted_g2form_vecs, 7, 3).numpy())
    print(f'G2 metrics computed... (shape: {predicted_g2metrics.shape})')

# Compute the MSE loss between computed and NN-predicted values
if not hp["metric"]:
    full_mse_loss = np.mean((test_link_outputs_tf.numpy() - predicted_g2forms) ** 2)
    test_outputs_vecs = np.array(form_to_vec(test_link_outputs_tf))
    vec_mse_loss = np.mean((test_outputs_vecs - predicted_g2form_vecs) ** 2)
else:
    full_mse_loss = np.mean((test_link_outputs_tf.numpy() - predicted_g2metrics) ** 2)
    test_outputs_vecs = np.array(form_to_vec(test_link_outputs_tf))
    vec_mse_loss = np.mean((test_outputs_vecs - predicted_g2metric_vecs) ** 2)
print(f'Test losses (full-form, dof-vector): {(full_mse_loss, vec_mse_loss)}')


/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: divide by zero encountered in det
  r = _umath_linalg.det(a, signature=signature)
/Users/ed/miniforge3/envs/cymetric/lib/python3.9/site-packages/numpy/linalg/linalg.py:2180: RuntimeWarning: invalid value encountered in det
  r = _umath_linalg.det(a, signature=signature)
pointgen:INFO:Vol_k: 5.0, Vol_cy: 287.45090973298755.


32/32 [==============================] - 0s 392us/step
G2 3-forms computed... (shape: (1000, 7, 7, 7))
Test losses (full-form, dof-vector): (0.004139248141103859, 0.006758321451522278)


In [11]:
# More loss testing
print(test_outputs_vecs.shape, predicted_g2form_vecs.shape)

# Define losses
mae = tf.keras.losses.MeanAbsoluteError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
mse = tf.keras.losses.MeanSquaredError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)
mape = tf.keras.losses.MeanAbsolutePercentageError(reduction=tf.keras.losses.Reduction.SUM_OVER_BATCH_SIZE)

# Compute losses
mae_loss = mae(test_outputs_vecs, predicted_g2form_vecs).numpy()
mse_loss = mse(test_outputs_vecs, predicted_g2form_vecs).numpy()
mape_loss = mape(test_outputs_vecs, predicted_g2form_vecs).numpy()

# Print losses
print("(MAE, MSE, MAPE):", (mae_loss, mse_loss, mape_loss))


(1000, 35) (1000, 35)
(MAE, MSE, MAPE): (0.036087938, 0.0067583215, 3760.761)


In [12]:
# Compute exterior derivative of 3-forms over the test data
# Retrieve the Kahler form for the test dataset
test_kahler_form = test_dataset.kahler_form

# Compute d\phi
dg2_3form = np.array(exterior_derivative(loaded_model.model, test_linkpts_tf, test_patch_idxs))
print(f'G2 3-form exterior derivatives computed... (shape: {dg2_3form.shape})')
###print(f'Non-zero elements: {np.sum(np.where(np.absolute(np.mean(dg2_3form[0],axis=0)) < 1e-5, 1, 0))} / {np.prod(dg2_3form.shape[2:])}')

# Compute omega ^ omega
omega_wedge_omega = np.array([wedge_product(test_kahler_form[idx], test_kahler_form[idx]) for idx in range(test_kahler_form.shape[0])])
print(f'\omega ^ \omega computed... (shape: {omega_wedge_omega.shape})')

# Check whether dg2_3form == omega_wedge_omega
print(f'Checking identity d\phi = \omega ^ \omega:\t{np.all(dg2_3form == omega_wedge_omega)}')


G2 3-form exterior derivatives computed... (shape: (1000, 7, 7, 7, 7))
\omega ^ \omega computed... (shape: (1000, 7, 7, 7, 7))
Checking identity d\phi = \omega ^ \omega:	False


In [ ]:
###manual checking -- deleteee
#np.sum(np.where(np.absolute(omega_wedge_omega) > 1e-8, 1, 0)),np.sum(np.where(np.absolute(dg2_3form) > 1e-8, 1, 0))
#print(np.min(dg2_3form), np.mean(dg2_3form), np.max(dg2_3form))
#print(np.min(omega_wedge_omega), np.mean(omega_wedge_omega), np.max(omega_wedge_omega))

for pt_idx in range(identity_test_size):
    print(np.mean(np.absolute(dg2_3form[0][pt_idx] - omega_wedge_omega[pt_idx])))
    ###reduce to non-zero?
    #--> consistently baddd


In [ ]:
# Compute \psi both ways ---> need the metric working first
# Compute as *\phi
from geometry.numerical_star_R7 import Hodge_Dual

##train 'good' model for metric
##computed metric & 3-form on test data
psi = np.array([Hodge_Dual(predicted_g2forms[pt_idx], predicted_g2metrics[pt_idx]) for pt_idx in range(testsize)])

# Compute as 0.5 * \omega ^ \omega + Im(\Omega) ^ d\theta



In [ ]:
########################################################################################

In [ ]:
###testinggg
from geometry.geometry import wedge_product, wedge_form2_with_form1
fake_batchsize = 10
'''#tf
fake_2_forms = tf.random.normal((fake_batchsize, 6, 6))
fake_2_forms = fake_2_forms - tf.transpose(fake_2_forms, perm=[0, 2, 1])
fake_1_forms = tf.random.normal((fake_batchsize, 7))
'''
#np
fake_2_forms = np.random.randn(fake_batchsize, 6, 6)
fake_2_forms_66 = fake_2_forms - np.transpose(fake_2_forms, axes=[0, 2, 1])
fake_2_forms_77 = np.pad(fake_2_forms_66, ((0,0), (0,1), (0,1)), mode='constant')
fake_1_forms = np.random.normal(size=(fake_batchsize, 7))
print(f'Data shapes: {fake_2_forms.shape}, {fake_1_forms.shape}')

# old functionality:
output_old = wedge_form2_with_form1(fake_2_forms_66, fake_1_forms)
output_new = np.array([wedge_product(fake_2_forms_77[idx], fake_1_forms[idx]) for idx in range(fake_1_forms.shape[0])])

# scale output_old to match output_new
output_old *= 3 ###why is there this factor of 3 difference?

print(f'Output shapes: {output_old.shape}, {output_new.shape}')
print(f'Matching?? --> {np.allclose(output_old, output_new)}')